# Rain Removal Video Processing with DIP R1/L2 and YOLO Tracking

This notebook processes the Rain1.mp4 video through:
1. DIP R1 pipeline (Rain Removal Enhancement)
2. DIP R2 pipeline (Frequency-based Rain Attenuation)
3. YOLO tracking on the original and both enhanced videos using `model.track`
4. Comparison visualization across all three variants


In [1]:
import os
import sys
import cv2
import shutil
from IPython.display import Video, HTML, display
from ultralytics import YOLO

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.dip.processors import build_DIP_pipeline, run_DIP_pipeline
from src.utils.video_utils import open_video, create_writer


## 📹 Video Display Notes

**If videos don't display:**
1. Make sure all cells above have been executed (variables must be defined)
2. The videos use `embed=True` which embeds them in the notebook
3. Large videos (40-79MB) may take 10-30 seconds to load
4. If your browser freezes, clear outputs: `Cell > All Output > Clear`
5. Re-run just the video display cell you want to see

**Memory management:**
- After viewing a video, clear its output to free memory
- You can re-run the cell anytime to view it again
- Alternatively, navigate to the `output/` folder and open MP4 files directly in a video player


## Setup Paths


In [2]:
# Define paths
project_root = "/Users/pete/Desktop/253_Project"
input_video = os.path.join(project_root, "videos/Rain/Rain1.mp4")
model_path = os.path.join(project_root, "models/best.pt")

# Create output directory
output_dir = os.path.join(project_root, "output")
os.makedirs(output_dir, exist_ok=True)

print(f"Input video: {input_video}")
print(f"YOLO model: {model_path}")
print(f"Output directory: {output_dir}")


Input video: /Users/pete/Desktop/253_Project/videos/Rain/Rain1.mp4
YOLO model: /Users/pete/Desktop/253_Project/models/best.pt
Output directory: /Users/pete/Desktop/253_Project/output


## Step 1: Apply DIP R1 and L2 Enhancements (Rain Removal)


In [3]:
def process_video_with_dip(input_video_path, output_video_path, dip_process_name):
    """Process a video through DIP pipeline and save the result."""
    print(f"Processing video with DIP {dip_process_name}...")
    dip_function = build_DIP_pipeline(dip_process_name)

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video: {input_video_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"  Video info: {width}x{height} @ {fps:.2f} fps, {total_frames} frames")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out.isOpened():
        raise RuntimeError(f"Failed to create writer: {output_video_path}")

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = run_DIP_pipeline(frame, dip_function)
        out.write(processed_frame)
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"  Processed {frame_count}/{total_frames} frames...")

    cap.release()
    out.release()
    print(f"DIP processing complete! Total frames: {frame_count}/{total_frames}")

# Process video with DIP R1 - output directly to MP4 for better smoothness
dip_r1_mp4 = os.path.join(output_dir, "Rain1_R1_processed.mp4")
process_video_with_dip(input_video, dip_r1_mp4, "R1")

# Process video with DIP R2 for comparison
dip_r2_mp4 = os.path.join(output_dir, "Rain1_R2_processed.mp4")
process_video_with_dip(input_video, dip_r2_mp4, "R2")


Processing video with DIP R1...
  Video info: 1670x1080 @ 30.00 fps, 1530 frames
  Processed 30/1530 frames...
  Processed 60/1530 frames...


KeyboardInterrupt: 

## Step 2: Run YOLO Detection on Original Video


In [ ]:
# Performance optimization settings
# Reduce imgsz for speed: 640 is ~2.25x faster than 960
# Trade-off: Lower imgsz = faster but potentially less accurate
imgsz = 640  # Options: 480 (fastest), 640 (balanced), 960 (most accurate)
device = None  # Auto-detect: None, or specify "cuda", "mps", "cpu"
half = False  # FP16 half precision (set True for GPU to speed up)

# Load model once and reuse for all videos (faster than reloading)
print("Loading YOLO model...")
model = YOLO(model_path)
device_label = device if device is not None else "auto"
print(f"Model loaded. Device: {device_label}, imgsz: {imgsz}")

In [ ]:
def run_yolo_on_video(
    input_video_path,
    output_video_path,
    model_path,
    model=None,
    imgsz=2560,
    device=None,
    half=False,
):
    """Run YOLO tracking on a video and save annotated frames with counting info."""

    video_name = os.path.basename(input_video_path)
    print(f"Running YOLO tracking on {video_name}...")

    yolo_model = model if model is not None else YOLO(model_path)

    runtime_device = device
    if runtime_device is None:
        import torch

        if torch.cuda.is_available():
            runtime_device = "cuda"
        elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            runtime_device = "mps"
        else:
            runtime_device = "cpu"

    print(f"  Using device: {runtime_device}, imgsz: {imgsz}, half: {half}")

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video: {input_video_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    print(f"  Video info: {width}x{height} @ {fps:.2f} fps, {total_frames} frames")

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out.isOpened():
        raise RuntimeError(f"Failed to create writer: {output_video_path}")

    counting_line_y = int(height * 0.65)
    counted_ids = set()
    vehicle_count = 0
    threshold = 55
    frame_count = 0
    total_detections = 0
    frame_counts = {}

    tracking_stream = yolo_model.track(
        source=input_video_path,
        imgsz=imgsz,
        conf=0.01,
        iou=0.4,
        verbose=False,
        max_det=300,
        stream=True,
        persist=True,
        device=runtime_device,
        half=half,
    )

    for frame_count, result in enumerate(tracking_stream, start=1):
        annotated_frame = result.plot(line_width=2, conf=True)
        boxes = result.boxes
        num_detections = len(boxes) if boxes is not None else 0
        total_detections += num_detections
        frame_counts[frame_count] = num_detections

        if boxes is not None and boxes.id is not None:
            xyxy = boxes.xyxy.cpu().numpy()
            track_ids = boxes.id.int().cpu().tolist()
            for (x1, y1, x2, y2), track_id in zip(xyxy, track_ids):
                cx = (x1 + x2) / 2.0
                cy = (y1 + y2) / 2.0
                if abs(cy - counting_line_y) < threshold and track_id not in counted_ids:
                    counted_ids.add(track_id)
                    vehicle_count += 1
                cv2.circle(annotated_frame, (int(cx), int(cy)), 4, (0, 255, 255), -1)
                cv2.putText(
                    annotated_frame,
                    f"ID {track_id}",
                    (int(cx) - 20, int(cy) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 255),
                    2,
                )

        cv2.line(annotated_frame, (0, counting_line_y), (width, counting_line_y), (0, 255, 0), 3)
        count_text = f"Vehicle Count: {vehicle_count}"
        cv2.putText(annotated_frame, count_text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
        out.write(annotated_frame)

        if frame_count % 30 == 0 or frame_count == total_frames:
            print(f"  Processed {frame_count}/{total_frames} frames...")

    out.release()
    print(
        f"YOLO tracking complete! Frames: {frame_count}/{total_frames}, Total detections: {total_detections}"
    )
    print(f"Vehicles counted crossing line: {vehicle_count}")
    return vehicle_count, frame_counts


## Step 3: Run YOLO Detection on Original + DIP-Enhanced Videos


In [ ]:
# Run YOLO tracking on original video - output directly to MP4
original_yolo_mp4 = os.path.join(output_dir, "Rain1_original_yolo.mp4")

original_count, original_frames = run_yolo_on_video(
    input_video,
    original_yolo_mp4,
    model_path,
    model=model,
    imgsz=imgsz,
    device=device,
    half=half,
)


In [ ]:
# Run YOLO tracking on DIP R1 enhanced video - output directly to MP4
processed_r1_yolo_mp4 = os.path.join(output_dir, "Rain1_enhanced_yolo.mp4")

r1_count, r1_frames = run_yolo_on_video(
    dip_r1_mp4,
    processed_r1_yolo_mp4,
    model_path,
    model=model,
    imgsz=imgsz,
    device=device,
    half=half,
)

# Run YOLO tracking on DIP R2 enhanced video - output directly to MP4
processed_r2_yolo_mp4 = os.path.join(output_dir, "Rain1_R2_processed_yolo.mp4")

r2_count, r2_frames = run_yolo_on_video(
    dip_r2_mp4,
    processed_r2_yolo_mp4,
    model_path,
    model=model,
    imgsz=imgsz,
    device=device,
    half=half,
)


## Vehicle Count Comparison


### Counting Results


In [ ]:
ORIGINAL_COUNT = original_count
print("=" * 60)
print("VEHICLE COUNTING RESULTS")
print("=" * 60)
print(f"Original Video:       {ORIGINAL_COUNT} vehicles counted")
print(f"DIP R1 Video:         {r1_count} vehicles counted")
print(f"DIP R2 Video:         {r2_count} vehicles counted")
print("-" * 60)
print(f"R1 - Original:        {r1_count - original_count:+} vehicles")
print(f"R2 - Original:        {r2_count - original_count:+} vehicles")
print(f"R2 - R1:              {r2_count - r1_count:+} vehicles")
print("=" * 60)
print("\nThe DIP enhancements improve visibility in different ways, so review")
print("both outputs alongside the original to decide which tracking result")
print("aligns best with your downstream requirements.")


In [ ]:
# Helper function for robust video display
try:
    import imageio_ffmpeg
except ImportError:
    print("Installing imageio-ffmpeg for video conversion...")
    %pip install -q imageio-ffmpeg
    import imageio_ffmpeg

def display_video_robust(video_path, width=960):
    """
    Display video with fallback options if embedding fails.
    Converts video to H.264 using ffmpeg (via imageio-ffmpeg) for browser compatibility.
    """
    from IPython.display import Video, FileLink, display, HTML
    import os
    import subprocess
    
    if not os.path.exists(video_path):
        print(f"❌ Video not found: {video_path}")
        print("   Run the processing cells above first.")
        return None

    # Get ffmpeg executable from imageio-ffmpeg
    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()
    
    # Output path for the converted video (browser compatible)
    base, ext = os.path.splitext(video_path)
    converted_path = f"{base}_h264.mp4"
    
    print(f"Processing video: {os.path.basename(video_path)}")
    
    # Convert to H.264 using ffmpeg
    # -y: overwrite output
    # -loglevel panic: suppress output
    # -vcodec libx264: use H.264 codec
    # -pix_fmt yuv420p: ensure compatibility
    # -acodec aac: audio codec (if audio exists)
    cmd = [
        ffmpeg_exe, "-y", "-loglevel", "panic",
        "-i", video_path,
        "-vcodec", "libx264",
        "-pix_fmt", "yuv420p",
        "-acodec", "aac",
        converted_path
    ]
    
    try:
        subprocess.run(cmd, check=True)
        
        if os.path.exists(converted_path):
            file_size_mb = os.path.getsize(converted_path) / (1024*1024)
            print(f"📹 Displaying: {os.path.basename(converted_path)} ({file_size_mb:.1f} MB)")
            print(f"   Loading... (may take 10-30 seconds for large files)")
            return Video(converted_path, embed=True, width=width, html_attributes="controls")
        else:
            print("⚠️ Conversion failed (output file not created). displaying original...")
    except Exception as e:
        print(f"⚠️ ffmpeg conversion failed: {e}")
        print("   Attempting to display original file...")
    
    # Fallback to original
    try:
        file_size_mb = os.path.getsize(video_path) / (1024*1024)
        print(f"📹 Video: {os.path.basename(video_path)} ({file_size_mb:.1f} MB)")
        return Video(video_path, embed=True, width=width, html_attributes="controls")
    except Exception as e:
        print(f"⚠️  Embedding failed: {e}")
        print(f"   Alternative: Click to download/open externally:")
        display(FileLink(video_path))
        return None

print("✓ Helper function loaded. Use: display_video_robust(video_path)")

## YOLO Tracking Results with Vehicle Counting

### YOLO on Original Video (with counting line and count overlay)


In [ ]:
display_video_robust(original_yolo_mp4)

### YOLO on DIP R1 Enhanced Video (with counting line and count overlay)


In [ ]:
display_video_robust(processed_r1_yolo_mp4)

### YOLO on DIP R2 Enhanced Video (with counting line and count overlay)



In [ ]:
display_video_robust(processed_r2_yolo_mp4)

## Summary

This notebook processed the Rain1.mp4 video through:

1. **DIP R1 Enhancement**: Applied Temporal median + Bilateral filtering in YCrCb color space
2. **DIP R2 Enhancement**: Ran a second DIP variant to preserve edges and frequency-based rain attenuation
3. **YOLO Tracking with `model.track()`**: Evaluated the original, DIP R1, and DIP R2 videos using the built-in tracker for persistent IDs
4. **Vehicle Counting & Comparison**: Counted line crossings for each variant and compared the resulting counts and visuals

### Key Features:
- **Built-in Tracking**: Relied on `model.track()` for ID persistence—no custom centroid tracker required
- **Better Codec**: Used mp4v codec directly instead of MJPG + ffmpeg conversion
- **Frame Preservation**: Processed every frame without skipping to maintain smooth playback
- **Proper FPS**: Ensured output videos maintain the source frame rate
- **Vehicle Counting**: Simple line-crossing algorithm using YOLO track IDs so each vehicle is counted only once

### How Vehicle Counting Works:
1. A horizontal green line is drawn roughly 2/3 down each frame
2. When a vehicle's center crosses this line, it is counted
3. YOLO track IDs ensure each vehicle is counted only once (55-pixel threshold)
4. The current count is displayed in the top-left corner of each frame

Both DIP R1 and L2 improve visibility in low-light conditions—review their side-by-side outputs to decide which enhancement best serves your downstream analytics.


In [ ]:
# Evaluation Metrics
# Note: Ground truth available in data/ground_truth.csv
# Rain1.mp4 ground truth: 47 vehicles

print("="*60)
print("VEHICLE COUNTING RESULTS")
print("="*60)
print(f"Original Video:       {original_count} vehicles")
print(f"DIP R1 Video:         {r1_count} vehicles")
print(f"DIP R2 Video:         {r2_count} vehicles")
print(f"Ground Truth:         47 vehicles")
print("="*60)
print("\nComparison with Ground Truth:")
print(f"Original - GT:        {original_count - 47:+} vehicles")
print(f"DIP R1 - GT:          {r1_count - 47:+} vehicles")
print(f"DIP R2 - GT:          {r2_count - 47:+} vehicles")
print("="*60)
